In [5]:
import os
import openpyxl
import cv2
import time
import datetime
from tqdm.notebook import tqdm

## 配置

In [6]:
output_txt_path = './mot_save/mot_results/output.txt'                              #FairMOT算法输出的outpu.txt路径
source_video_path = '/home/swjtu/dataset/武侯高级中学监控视频/D14_20210929084614.mp4' #原视频路径

source_video_camera_id = 14                                                        #原视频相机序号
source_video_seq_id = 2                                                            #原视频片段序号
source_video_start_time = "20210929084615"                                         #原视频真实起始日期时间

source_video_frame_rate = 25.01                                                    #原视频帧率
detection_frame_rate = 25.01                                                       #MOT算法检测帧率
crop_frame_interval = 25                                                           #间隔多少帧截取一次行人图像
ignore_pids = []                                                                   #忽略行人ID
min_box_area = -1                                                                  #最小行人框大小（在FairMOT中设置即可）

save_crops_path = './mot_save/crops/'                                              #裁剪图片保存目录
save_excel_path = "./mot_save/D14_20210929084614_5分钟数据标注.xlsx"                 #excel保存目录

## 加载MOT输出结果

In [7]:
output = []
with open(output_txt_path,'r') as f:
    for line in f.readlines():
        output.append(line.split(',')[:7])

## 根据Output打开原视频截取行人图片

In [8]:
cap = cv2.VideoCapture(source_video_path)
cur_frame = 1
frame_ratio = source_video_frame_rate / detection_frame_rate
index = 0
len_output = len(output)
pbar = tqdm(total=len_output)
pbar.set_description('Processing:')
while index < len_output:
    ret, img = cap.read()   
    if ret == False or img is None: break
    while index < len_output and cur_frame == int(output[index][0]):
        frame, pid, x, y, w, h = [int(float(i)) for i in output[index][:-1]]
        index += 1
        pbar.update(1)
        if (cur_frame-1) % crop_frame_interval != 0 or pid in ignore_pids or (min_box_area!=-1 and w*h<min_box_area): 
            continue 
        
        time = datetime.datetime.strptime(source_video_start_time, "%Y%m%d%H%M%S") + datetime.timedelta(seconds=round(frame/detection_frame_rate))
        fileName = "%04d_c%02ds%02d_%04d_%s.jpg" % (pid, source_video_camera_id, source_video_seq_id, round(frame * frame_ratio)
                                                    , time.strftime("%Y%m%d%H%M%S"))
        if x*y*w*h > 0:
            cv2.imwrite(os.path.join(save_crops_path, fileName), img[y:y+h, x:x+w])
        
    cur_frame += 1

  0%|          | 0/3225 [00:00<?, ?it/s]

## 创建Excel并写入数据

In [13]:
wb = openpyxl.Workbook()
ws = wb['Sheet']
ws.append(['人员ID', '进入时间', '离开时间', '进入帧号', '离开帧号', '摄像头ID', '视频片段序号'])

idMap = {}
for frame,pid,x1,y1,w,h,score in output:
    if pid in ignore_pids or (min_box_area!=-1 and float(w)*float(h)<min_box_area): continue
    if pid not in idMap:
        idMap[pid] = [int(frame), int(frame)] # in_frame, out_frame
    else:
        idMap[pid][1] = int(frame)
for pid in idMap:
    in_time, out_time = [ "%02d:%02d:%02d" % (time//3600, time%3600//60, time%60) for time in [ frame // detection_frame_rate for frame in idMap[pid]] ]
    ws.append([pid, in_time, out_time, idMap[pid][0], idMap[pid][1], source_video_camera_id, source_video_seq_id])
wb.save(save_excel_path)